In [24]:
import sys 
sys.path.append('../')
import jax 
import jax.numpy as jnp 
import numpy as np
from jaxlie import SE3, SO3
from jax.random import normal
from jax import grad, hessian, vmap, pmap
from jax.flatten_util import ravel_pytree
from jax.lax import scan
from functools import partial
import jax.random as jax_random
import matplotlib.pyplot as plt 
from IPython.display import clear_output
from plyfile import PlyData
import open3d as o3d 
import trimesh as tm
import trimesh
import time
import jaxopt
import polyscope as ps


import meshcat
import meshcat.geometry as mc_geom

from ergodic_mmd.aug_lagrange_jaxopt import AugmentedLagrangeSolver
# from ergodic_mmd.aug_lagrange_solver import AugmentedLagrangeSolver

import adam
from adam.jax import KinDynComputations

In [2]:
from robomeshcat import Object, Robot, Scene
from example_robot_data.robots_loader import PandaLoader
from pathlib import Path

"Create a scene that stores all objects and robots and has rendering capability"
scene = Scene()

robot = Robot(urdf_path=PandaLoader().df_path, mesh_folder_path=Path(PandaLoader().model_path).parent.parent)
scene.add_robot(robot)
"Render the initial scene"
scene.render()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [29]:
# dataset = o3d.data.BunnyMesh()
# plydata = PlyData.read(dataset.path)
# plydata = PlyData.read('../assets/sphere.ply')
# verts = np.vstack((
#     plydata['vertex']['x'], # offset mesh location by half a meter
#     plydata['vertex']['y'],
#     plydata['vertex']['z']
# )).T
# faces = np.array(np.vstack(plydata['face']['vertex_indices']))

# mesh = tm.Trimesh(vertices=verts, faces=faces)
# mesh.apply_scale(1/10)
# mesh.apply_translation(np.array([0.5,0.,0.25]))

# exp_obj = trimesh.exchange.obj.export_obj(mesh)
# mc_obj = mc_geom.ObjMeshGeometry.from_stream(trimesh.util.wrap_as_stream(exp_obj))
# mc_obj = Object(mc_obj, name='red_sphere', opacity=0.5, color=[1., 0., 0.])
# scene.add_object(mc_obj)


# num_points = 100  # Change this number based on your requirement
# points, face_indices = tm.sample.sample_surface(mesh, num_points)

# info_distr = lambda x: 1.0#jnp.exp(-20*(x[0]-0.4)**2 - 20*(x[2]-0.3)**2)
# P_XI = vmap(info_distr, in_axes=(0,))(points)
# P_XI = P_XI/jnp.sum(P_XI)

_point = jnp.array([0.5,0.,0.25])
_normal = jnp.array([-1,0,1])
_normal = _normal/jnp.linalg.norm(_normal)
h = 0.2
args = {'h' : h, 
        'point' : _point, 
        'normal' : _normal}

In [4]:
model_path = "../assets/panda.urdf"
# The joint list
joints_name_list = [
    'panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 
    'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_joint8'
]

kinDyn = KinDynComputations(model_path, joints_name_list)
# kinDyn.set_frame_velocity_representation(adam.Representations.BODY_FIXED_REPRESENTATION)
w_H_ee = kinDyn.forward_kinematics_fun("panda_grasptarget")
w_H_b = np.eye(4) # base frame 

q0 = jnp.array([0,0,0,-1.57079, 0, 1.57079, -0.7853, 0.0])

# w_H_ee(w_H_b, q0)

Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link0']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link1']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link2']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link4']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link5']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link6']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link7']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_hand']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_leftfinger']/collision[1]
Unknown tag "contact" in /robot[@name='panda']/link[@name='panda_leftfinger']
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_rightfinger']/collision[1]
Unknown tag "contact" in /robot[@name='pan

In [30]:
def get_fk(q):
    return SE3.from_matrix(w_H_ee(w_H_b, q))

def RBF_kernel(x, xp, h=0.01):
    return jnp.exp(
        -jnp.sum((x-xp)**2)/h
    )

def camera_view_penalty(g, v, h=0.01):
    p = g.translation()
    rot = g.rotation()
    w = rot.apply(jnp.array([0.,0.,1.]))
    root, norm  = jnp.split(v, 2)
    return -jnp.exp(-jnp.sum((root-p)**2)/h - jnp.sum((-norm-w)**2)/h)

# def RBF_kernel(g, v, h=0.01):
#     p = g.translation()
#     rot = g.rotation()
#     w = rot.apply(jnp.array([0.,0.,1.]))
#     root, norm  = jnp.split(v, 2)
#     return jnp.exp(
#         -jnp.sum((root-p)**2)/h #- 1e-1*jnp.sum((norm-w)**2)
#     ) * jnp.exp(
#         -10*jnp.sum((-norm-w)**2)
#     )

def create_kernel_matrix(kernel):
    return vmap(vmap(kernel, in_axes=(0, None, None)), in_axes=(None, 0, None))

camera_view_matrix = create_kernel_matrix(camera_view_penalty)
KernelMatrix = create_kernel_matrix(RBF_kernel)
def diff_ik_loss(params, args):
    X = params['X']
    T = X.shape[0]
    h = args['h']
    point    = args['point']
    normal     = args['normal']
    v = jnp.hstack([point, normal])
    g = vmap(get_fk)(X)
    _value = vmap(camera_view_penalty, in_axes=(0, None, None))(g, v, h)
    return jnp.mean(_value) + jnp.mean(jnp.square(X[1:] - X[:-1])) + jnp.mean(jnp.square(X-q0))
    

def eq_constr(params, args):
    X = params['X']
    return jnp.array(0.)

def ineq_constr(params, args):
    X = params['X']
    return jnp.array(0.)


In [31]:
T = 100
X = jnp.linspace(q0, q0+0.1, num=T)

params = {'X' : X}

# solver = AugmentedLagrangeSolver(params, diff_ik_loss, eq_constr, ineq_constr, max_stepsize=3e-1, args=args)
solver = AugmentedLagrangeSolver(params, diff_ik_loss, eq_constr, ineq_constr, args=args)


In [32]:
solver.solve(eps=1e-3)

/home/anon/miniconda3/lib/python3.10/site-packages/jax/_src/core.py:691: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(
/home/anon/miniconda3/lib/python3.10/site-packages/jax/_src/core.py:691: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(


0 0.045813209289389846
1 0.04595356834473621
2 0.046094517133772266
3 0.04623605393827493
4 0.046378179558371355
5 0.04652089476339365
6 0.04666420027497419
7 0.046808096765694494
8 0.04695258485780794
9 0.04709766512194066
10 0.047243338075769754
11 0.047389604182678624
12 0.047536463850389225
13 0.047683917429570885
14 0.04783196521242561
15 0.04798060743124961
16 0.04812984425697074
17 0.04827967579766177
18 0.04843010209702923
19 0.048581123132877546
20 0.048732738815548446
21 0.04888494898633535
22 0.0490377534158724
23 0.04919115180249837
24 0.04934514377059475
25 0.049499728868898425
26 0.04965490656878821
27 0.04981067626254567
28 0.0499670372615895
29 0.05012398879468402
30 0.05028153000612094
31 0.050439659953874894
32 0.0505983776077324
33 0.05075768184739408
34 0.05091757146055022
35 0.05107804514092965
36 0.05123910148632177
37 0.051400738996571764
38 0.051562956071549
39 0.05172575100908869
40 0.05188912200290661
41 0.05205306714048722
42 0.052217584400945004
43 0.0523826

In [33]:
sol = solver.solution
X = sol['X']
vertices = vmap(get_fk)(X).translation().astype(np.float32).T
scene.vis['lines_segments'].set_object(mc_geom.Line(mc_geom.PointsGeometry(vertices), mc_geom.MeshLambertMaterial(color=0xff0000)))
ee_frame = scene.vis['ee']
ee_frame.set_object(mc_geom.triad(scale=0.2))
for q in X:
    _tf = np.array(get_fk(q).as_matrix())
    ee_frame.set_transform(_tf)
    for i, jt in enumerate(q):
        robot[i] = jt
        time.sleep(0.01)
    scene.render()

In [16]:
# ps.set_up_dir("z_up")
# ps.set_front_dir("neg_y_front")
ps.init()


ps_mesh = ps.register_surface_mesh("bunny", mesh.vertices, mesh.faces)
# ps_points = ps.register_point_cloud("sampled points", args['points'][:,:3])
# ps_mesh.add_scalar_quantity("face vals",vmap(info_distr, in_axes=(0,))(mesh.vertices))

# ps_points.add_scalar_quantity("results", args['P_XI'])

# ps_bunny_mesh.set_transparency(0.8)
# ps_bunny_mesh.add_scalar_quantity('info_distr', mesh_func.func_vals, defined_on='vertices', cmap='blues')

ps_traj         = ps.register_curve_network("trajectory", vmap(get_fk)(X).translation() , edges="line")


for _ in range(1000):
    solver.solve(eps=1e-5, max_iter=10)
    sol = solver.solution
    X = sol['X']

    ps_traj.update_node_positions(vmap(get_fk)(X).translation() )
    # ps_traj.add_vector_quantity("vec img", X[:,:3], enabled=True)
    ps.frame_tick()

    time.sleep(0.001)

iter  0  loss  0.9522419  grad l2 norm  0.107212
iter  1  loss  0.94674367  grad l2 norm  0.116121985
iter  2  loss  0.94212383  grad l2 norm  0.12298891
iter  3  loss  0.9378236  grad l2 norm  0.1289697
iter  4  loss  0.9336627  grad l2 norm  0.13444118
iter  5  loss  0.929548  grad l2 norm  0.13959293
iter  6  loss  0.9254174  grad l2 norm  0.14452823
iter  7  loss  0.92123663  grad l2 norm  0.14927922
iter  8  loss  0.9169976  grad l2 norm  0.15384422
iter  9  loss  0.91270345  grad l2 norm  0.15821983
unsuccessful, tol:  0.15821983
iter  0  loss  0.908359  grad l2 norm  0.16240726
iter  1  loss  0.90396935  grad l2 norm  0.16641097
iter  2  loss  0.8995379  grad l2 norm  0.17023695
iter  3  loss  0.895068  grad l2 norm  0.17389196
iter  4  loss  0.89056176  grad l2 norm  0.1773828
iter  5  loss  0.88602084  grad l2 norm  0.18071574
iter  6  loss  0.8814468  grad l2 norm  0.18389635
iter  7  loss  0.87684125  grad l2 norm  0.18692926
iter  8  loss  0.87220544  grad l2 norm  0.189818

In [ ]:
solver.solve(eps=1e-3)

In [22]:
sol = solver.solution

In [ ]:
X = sol['X']

ps.init()

ps_bunny_mesh = ps.register_surface_mesh("bunny", mesh.vertices, mesh.faces)
# ps_bunny_mesh.set_transparency(0.8)
# ps_bunny_mesh.add_scalar_quantity('info_distr', mesh_func.func_vals, defined_on='vertices', cmap='blues')

ps_traj         = ps.register_curve_network("trajectory", vmap(get_fk)(X).translation() , edges="line")

ps.show()